# 构建检索问答链

我们已经介绍了如何根据自己的本地知识文档，搭建一个向量知识库。 在接下来的内容里，我们将使用搭建好的向量数据库，对 query 查询问题进行召回，并将召回结果和 query 结合起来构建 prompt，输入到大模型中进行问答。   

## 1. 加载向量数据库

首先，我们加载在前一章已经构建的向量数据库。注意，此处你需要使用和构建时相同的 Emedding。

设置embedding - zhipu

In [ ]:
# import sys
# sys.path.append("../C3 搭建知识库") # 将父目录放入系统路径中

from langchain.vectorstores.chroma import Chroma
from dotenv import load_dotenv, find_dotenv
import os
# # 从环境变量中加载你的 API_KEY
# _ = load_dotenv(find_dotenv())    # read local .env file
# zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']

# 定义持久化目录
persist_directory = '../chroma-vmax'

# # 创建嵌入模型
# from langchain_community.embeddings import ZhipuAIEmbeddings

# zhipu_embed = ZhipuAIEmbeddings(
#     model="embedding-2",
#     api_key=zhipuai_api_key
# )

from langchain_community.embeddings import OllamaEmbeddings
emb_bgem3 = OllamaEmbeddings(base_url='http://localhost:11434',model="bge-m3:latest")

try:
    # 加载持久化的 Chroma 向量数据库
    vectordb = Chroma(
        persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
        collection_name="vmax-s",
        embedding_function=emb_bgem3
    )
    print("向量数据库已成功加载。")
except Exception as e:
    print(f"加载向量数据库时发生错误: {e}")

milvus

In [29]:
from langchain_community.vectorstores import Milvus
emb_bgem3 = OllamaEmbeddings(base_url='http://localhost:11434', model="bge-m3:latest")

# Milvus 连接参数
vectordb = Milvus(
        embedding_function=emb_bgem3,
        collection_name="Vmaxs",  # Milvus 集合名称
        connection_args={
            "host": "192.168.0.188",  # Milvus 服务器地址
            "port": "19530",  # Milvus 默认端口
        },
    )

C:\Users\will\AppData\Local\Temp\ipykernel_28400\4181978582.py:5: LangChainDeprecationWarning: The class `Milvus` was deprecated in LangChain 0.2.0 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-milvus package and should be used instead. To use it run `pip install -U :class:`~langchain-milvus` and import as `from :class:`~langchain_milvus import MilvusVectorStore``.
  vectordb = Milvus(


In [30]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

AttributeError: 'Milvus' object has no attribute '_collection'

In [ ]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

In [31]:
question = "VMAX上网日志业务是什么？"
docs = vectordb.similarity_search(question,k=5)
print(f"检索到的内容数：{len(docs)}")

检索到的内容数：5


打印一下检索到的内容

In [32]:
for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")

检索到的第0个内容: 
 4 功能本章包含如下主题：上网日志保存
10
上网日志查询
10
上网日志批量导入查询
11
日志管理
11
账号管理
11
角色管理
12
资源监控
12
告警管理
12
省级网关对接
12
拨测结果自动比对功能
12
NAT日志入库功能
13
北向接口
13
上网日志历史查询
13
云化上网日志XDR查询
13
以下介绍ZXVMAX-S的主要的功能。
4.1 上网日志保存支持使用Gbase数据库或HDFS保存上网日志。上网日志保存时间可配置，最短保存7天时间，最长可保存一年时间。支持自动清理超过保存时间的上网日志。
4.2 上网日志查询可通过web界面指定查询条件，查询用户上网日志。支持的查询条件：时间范围+公网IP
时间范围+目的IP
时间范围+MSISDN
时间范围+IMSI
时间范围+URL
支持组合以上五种基础条件，或在基础条件上增加指定公网端口、目的端口，做更精确的查询。
10
SJ-20220623151803-017|2023-03-30（R1.0）
-----------------------------------------------------
检索到的第1个内容: 
 4功能
4.11 NAT日志入库功能
ZXVMAX-S支持将NAT日志保存到GP数据库，可通过公网IP或目的IP查询NAT日志。
4.12 北向接口支持中移规范北向接口，中移集团查询平台下发查询命令，上网日志系统将查询结果以文件方式上传至查询平台。包括云化核心网的4G/5G上网日志查询、4G/5G控制面用户面XDR查询、
4G/5G软采XDR查询、VoLTEXDR查询、5G消息XDR查询，以及原始码流查询功能。
4.13 上网日志历史查询当现网使用两个库分别存储2个月数据和6个月数据时，使用历史数据查询操作查询6个月库。
4.14 云化上网日志XDR查询上网日志系统支持云化上网日志XDR查询，包括云化核心网的4G/5G控制面用户面XDR查询、
4G/5G软采XDR查询、VoLTEXDR查询、5G消息XDR查询，以及原始码流查询功能。
SJ-20220623151803-017|2023-03-30（R1.0）
13
------------------------------------------

## 2. 创建一个 LLM

在这里，我们调用 OpenAI 的 API 创建一个 LLM，当然你也可以使用其他 LLM 的 API 进行创建

In [ ]:
# # 从环境变量中加载你的 API_KEY
# _ = load_dotenv(find_dotenv())    # read local .env file
# zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']

# from langchain_openai import ChatOpenAI
# zhipuai_llm = ChatOpenAI(
#     temperature=0,
#     model="glm-4",
#     openai_api_key=zhipuai_api_key,
#     openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
# )

In [ ]:
# from langchain_community.llms import Ollama

# llm_deepseek = Ollama(base_url='http://localhost:11434', model='deepseek-r1:1.5b', temperature=0.1)

In [33]:

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_ollama import OllamaLLM

llm_deepseek = OllamaLLM(base_url='http://localhost:11434', model='deepseek-r1:1.5b', temperature=0.1, streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()])

In [34]:
# from langchain_deepseek import ChatDeepSeek
# _ = load_dotenv(find_dotenv())    # read local .env file
# deepseek_api_key = os.environ['DEEPSEEK_API_KEY']
# llm_deepseek = ChatDeepSeek(
#     model="deepseek-reasoner",
#     temperature=0,
#     max_tokens=None,
#     timeout=None,
#     # max_retries=2,
#     api_key=deepseek_api_key,
#     # other params...
# )

In [35]:
# from langchain_deepseek import ChatDeepSeek

# _ = load_dotenv(find_dotenv())    # read local .env file
# deepseek_api_key = os.environ['DEEPSEEK_API_KEY']

# from langchain_openai import ChatOpenAI
# zhipuai_llm = ChatOpenAI(
#     temperature=0,
#     model="deepseek-chat",
#     openai_api_key=deepseek_api_key,
#     openai_api_base="https://api.deepseek.com"
# )

## 3. 构建检索问答链

prompts

In [36]:
from langchain.prompts import PromptTemplate

template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)


#### 创建一个基于模板的检索链： 基础检索版本

In [41]:
from langchain.chains import RetrievalQA

# 基础检索
base_retriever = vectordb.as_retriever(search_kwargs={"k": 10})
base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    # metadata_filter={"source": "权威文档.pdf"}  # 元数据过滤
)

qa_chain = RetrievalQA.from_chain_type(llm_deepseek,
                                       retriever=base_retriever,
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})


In [42]:
question_1 = "什么是安全加固？"
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

<think>
嗯，用户问的是“什么是安全加固？”我得先理解这个问题。安全加固应该是指在网络安全方面进行增强措施的技术或方法吧？可能是在现有的网络中添加额外的保护机制。

首先，我需要回忆一下常见的安全加固技术。比如，防火墙、入侵检测系统（IDS）、加密通信等等。这些都是常用的手段来提升网络的安全性。

然后，我要考虑用户的需求是什么。他们可能在学习网络安全，或者在实际工作中需要确保自己的网络安全。所以，我应该详细解释这些方法，并给出一些具体的例子和应用场景。

另外，用户的问题中提到了一些具体的命令和步骤，比如配置文件、防火墙设置等。这些都是具体的实施方式，可能需要详细说明每一步的作用和效果。

我还得注意，用户可能对某些术语不太熟悉，比如“入侵检测系统”或者“加密通信”，所以最好用简单明了的语言解释清楚，避免他们感到困惑。

最后，总结一下安全加固的重要性，强调它在保护网络免受攻击中的关键作用。这样用户就能明白为什么这些方法是必要的，并且如何应用它们来提升网络安全水平。
</think>

**什么是安全加固？**

安全加固是指通过增强或补充现有网络设备和技术的防护措施，以提高网络的安全性、抗干扰性和防御能力。它通常包括但不限于以下内容：

1. **防火墙（Firewall）**：通过配置防火墙规则，限制外部流量进入或离开特定的网络区域，防止未经授权的访问。

2. **入侵检测系统（IDS）**：部署 IDS来实时监控网络流量，识别潜在的攻击行为并及时报告。

3. **加密通信**：使用加密技术（如SSL、HTTPS）确保数据在传输过程中的安全性，防止被窃取或篡改。

4. **访问控制机制（ACM）**：通过配置访问权限管理规则，限制特定用户和资源的访问权限，防止未经授权的访问。

5. **网络审计工具**：部署网络审计工具（如NAT、NAT-AE等），实时监控网络流量，并报告异常情况。

6. **入侵检测与响应（IDR）系统**：通过IDR系统，及时发现和处理网络攻击行为，减少潜在损失。

7. **虚拟专用网路（VPN）**：使用VPN技术，保护敏感数据在非目标网络上传输，防止未经授权的访问。

8. **加密链路**：通过加密链路（如IPsec、TLS）确保数据在传输过程中的安全性，防止被截获或篡改。

9. **多因素认证（MFA）**：通

In [ ]:
question_1 = "严威是谁？"
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

In [40]:
question_1 = "VMAX是什么？"
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

<think>
嗯，用户的问题是关于VMAX是什么。我需要先理解VMAX是什么，然后回答他的问题。

首先，我知道VMAX是一个技术术语，可能与无线通信有关。它可能指的是VLAN（虚拟局域网）或者VLAN的管理方法。VLAN通常用于分层网络中，将物理连接的设备分成不同的网络段，这样可以提高网络的灵活性和安全性。

然后，我想到VMAX可能是指VLAN管理的一种高级技术，比如VLAN管理协议、管理策略或优化方法。这可能涉及到如何高效地管理VLAN，以提升网络性能。

接下来，我需要确认是否有相关的资料或者文档提到过VMAX的具体含义。如果有的话，应该提供详细的解释；如果没有，我可以给出一个通用的定义，并说明它在无线通信中的应用。

最后，我要确保回答清晰明了，可能分点列出不同的方面来解释VMAX，比如技术背景、管理策略等，这样用户可以更全面地理解它的概念。
</think>

VMAX（Virtual Local Area Network Management）是一个与局域网（Local Area Network, LAN）相关的术语，通常指虚拟局域网的管理方法或协议。它在无线通信和网络管理中具有重要应用。

### VMAX 的定义：
VMAX 是一种通过虚拟化的方式来管理实际物理连接的设备，以提高网络的灵活性、安全性以及性能。VLAN（虚拟局域网）是将物理连接的设备分成不同的网络段，通常用于分层网络中的分组技术。

### VMAX 的应用：
1. **虚拟化管理**：VMAX 通过虚拟化的方式管理实际的物理设备，例如通过协议或工具来实现对设备的访问和配置。
2. **性能优化**：通过管理虚拟局域网，可以更好地优化网络性能，提升数据传输效率和网络稳定性。
3. **安全性**：虚拟化管理可以增强网络的安全性，防止未经授权的数据泄露和攻击。

### VMAX 的技术背景：
VMAX 通常基于以下技术：
- **VLAN（虚拟局域网）**：将物理连接的设备分成不同的虚拟段，用于分层网络中的分组。
- **管理协议**：例如，通过协议如VLAN Manager、VLAN Optimizer等来实现对虚拟局域网的管理和优化。

### VMAX 的应用场景：
1. **移动通信网络**：在移动通信中，VMAX 可以用于管理设备之间的连接和数据传输，提升移动网络的性能。

### 创建一个基于模板的检索链： rerank检索版本

In [ ]:
# # 测试API

# import cohere
# # co = cohere.ClientV2()
# co = cohere.Client(api_key="Tahx1eySFbKvu9sTyTXrRLf59la3ZUG9vy02stRZ")
# docs = [
#     "Carson City is the capital city of the American state of Nevada.",
#     "The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean. Its capital is Saipan.",
#     "Capitalization or capitalisation in English grammar is the use of a capital letter at the start of a word. English usage varies from capitalization in other languages.",
#     "Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district.",
#     "Capital punishment has existed in the United States since beforethe United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states.",
# ]
# response = co.rerank(
#     model="rerank-v3.5",
#     query="What is the capital of the United States?",
#     documents=docs,
#     top_n=3,
# )
# print(response)



cohere

In [ ]:
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
# rerank检索
# Cohere Rerank配置
import cohere
cohere_client = cohere.Client(api_key="Tahx1eySFbKvu9sTyTXrRLf59la3ZUG9vy02stRZ")

compressor = CohereRerank(
    client=cohere_client,
    top_n=5,
    model="rerank-multilingual-v3.0"  # 支持多语言的新版本
)

base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    # metadata_filter={"source": "权威文档.pdf"}  # 元数据过滤
)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)


qa_chain = RetrievalQA.from_chain_type(
    llm_deepseek,
    retriever=compression_retriever,  # 替换为压缩检索器
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": QA_CHAIN_PROMPT,
        # "llm_kwargs": {"max_length": 300}  # 新增输出长度限制
    }
)


jina

In [ ]:
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors import JinaRerank  # 使用Jina的rerank组件

# Jina Rerank配置
JINA_API_KEY = "jina_63bb115e2d5f42d581f42643294792b5CE4nrEINMDcT4vJZJaSLcr5tkbIB"  # 替换为你的Jina API密钥

compressor = JinaRerank(
    jina_api_key=JINA_API_KEY,
    top_n=3,
    model="jina-reranker-v2-base-multilingual"  # Jina的多语言rerank模型[5](@ref)
)

base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    # metadata_filter={"source": "权威文档.pdf"}  # 元数据过滤
)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)


qa_chain = RetrievalQA.from_chain_type(
    llm_deepseek,
    retriever=compression_retriever,  # 替换为压缩检索器
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": QA_CHAIN_PROMPT,
        # "llm_kwargs": {"max_length": 300}  # 新增输出长度限制
    }
)

Xinference部署的rerank模型

In [ ]:
from typing import List, Dict, Any
from langchain.retrievers.document_compressors import BaseDocumentCompressor
from langchain.schema import Document
import requests

class XinferenceRerank(BaseDocumentCompressor):
    def __init__(self, endpoint: str, model_uid: str, top_n: int = 5):
        self.endpoint = endpoint
        self.model_uid = model_uid
        self.top_n = top_n

    def compress_documents(
        self,
        documents: List[Document],
        query: str,
        **kwargs: Any,
    ) -> List[Document]:
        # 转换文档格式
        docs_text = [doc.page_content for doc in documents]
        
        # 调用 Xinference API（参考网页1）
        response = requests.post(
            f"{self.endpoint}/v1/rerank",
            json={
                "model": self.model_uid,
                "query": query,
                "documents": docs_text
            }
        )
        response.raise_for_status()
        results = response.json()["results"]
        
        # 按分数排序并截取 top_n
        sorted_indices = sorted(range(len(results)), key=lambda i: results[i]["relevance_score"], reverse=True)
        top_indices = sorted_indices[:self.top_n]
        
        return [documents[i] for i in top_indices]

# 新 Xinference 配置（网页1/5）
xinference_rerank = XinferenceRerank(
    endpoint="http://localhost:5544",  # 根据实际部署调整（网页1/2/3）
    model_uid="bce-reranker-base_v1",  # 与注册的 model_uid 一致
    top_n=5
)

# 更新压缩检索器
compression_retriever = ContextualCompressionRetriever(
    base_compressor=xinference_rerank,
    base_retriever=base_retriever
)



## 4.检索问答链效果测试

In [ ]:
question_1 = "VMAX是什么？"
question_2 = "严威是谁？"

### 4.1 基于召回结果和 query 结合起来构建的 prompt 效果

In [ ]:
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

In [ ]:
result = qa_chain({"query": question_2})
print("大模型+知识库后回答 question_2 的结果：")
print(result["result"])

### 4.2 大模型自己回答的效果

In [ ]:
prompt_template = """请回答下列问题:
                            {}""".format(question_1)

### 基于大模型的问答
llm_deepseek.predict(prompt_template)

In [ ]:
prompt_template = """请回答下列问题:
                            {}""".format(question_2)

### 基于大模型的问答
llm_deepseek.predict(prompt_template)

> ⭐ 通过以上两个问题，我们发现 LLM 对于一些近几年的知识以及非常识性的专业问题，回答的并不是很好。而加上我们的本地知识，就可以帮助 LLM 做出更好的回答。另外，也有助于缓解大模型的“幻觉”问题。

## 5. 添加历史对话的记忆功能-v2025

现在我们已经实现了通过上传本地知识文档，然后将他们保存到向量知识库，通过将查询问题与向量知识库的召回结果进行结合输入到 LLM 中，我们就得到了一个相比于直接让 LLM 回答要好得多的结果。在与语言模型交互时，你可能已经注意到一个关键问题 - **它们并不记得你之前的交流内容**。这在我们构建一些应用程序（如聊天机器人）的时候，带来了很大的挑战，使得对话似乎缺乏真正的连续性。这个问题该如何解决呢？


### 5.1 记忆（Memory）

在本节中我们将介绍 LangChain 中的储存模块，即如何将先前的对话嵌入到语言模型中的，使其具有连续对话的能力。我们将使用 `ConversationBufferMemory` ，它保存聊天消息历史记录的列表，这些历史记录将在回答问题时与问题一起传递给聊天机器人，从而将它们添加到上下文中。

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

关于更多的 Memory 的使用，包括保留指定对话轮数、保存指定 token 数量、保存历史对话的总结摘要等内容，请参考 langchain 的 Memory 部分的相关文档。

### 5.2 对话检索链（ConversationalRetrievalChain）

对话检索链（ConversationalRetrievalChain）在检索 QA 链的基础上，增加了处理对话历史的能力。

它的工作流程是:
1. 将之前的对话与新问题合并生成一个完整的查询语句。
2. 在向量数据库中搜索该查询的相关文档。
3. 获取结果后,存储所有答案到对话记忆区。
4. 用户可在 UI 中查看完整的对话流程。

这种链式方式将新问题放在之前对话的语境中进行检索，可以处理依赖历史信息的查询。并保留所有信
息在对话记忆中，方便追踪。

接下来让我们可以测试这个对话检索链的效果：

使用上一节中的向量数据库和 LLM ！首先提出一个无历史对话的问题“这门课会学习 Python 吗？”，并查看回答。

v2024

In [ ]:
from langchain.chains import ConversationalRetrievalChain

retriever=vectordb.as_retriever()

qa = ConversationalRetrievalChain.from_llm(
    llm_deepseek,
    retriever=retriever,
    memory=memory
)
question = "什么是安全加固？"
result = qa({"question": question})
print(result['answer'])

V2025

不带memory

In [ ]:
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
# rerank检索
# Cohere Rerank配置
import cohere
cohere_client = cohere.Client(api_key="Tahx1eySFbKvu9sTyTXrRLf59la3ZUG9vy02stRZ")
# compressor = CohereRerank(
#     client=cohere_client,
#     top_n=5  # 保留Top5相关文档
# )
compressor = CohereRerank(
    client=cohere_client,
    top_n=5,
    model="rerank-multilingual-v3.0"  # 支持多语言的新版本
)

base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    metadata_filter={"source": "权威文档.pdf"}  # 元数据过滤
)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)

# qa_chain = RetrievalQA.from_chain_type(zhipuai_llm,
#                                        retriever=base_retriever,
#                                        return_source_documents=True,
#                                        chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

qa_chain = RetrievalQA.from_chain_type(
    llm_deepseek,
    retriever=compression_retriever,  # 替换为压缩检索器
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": QA_CHAIN_PROMPT,
        # "llm_kwargs": {"max_length": 300}  # 新增输出长度限制
    }
)

result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])


新增MEMORY

In [ ]:
from langchain.chains import ConversationalRetrievalChain

from langchain.prompts import PromptTemplate

QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["chat_history", "question", "context"],
    template="""
    你是一个专业的问答助手。请根据对话历史和提供的上下文回答问题。
    
    历史对话：
    {chat_history}
    
    上下文：
    {context}
    
    问题：{question}
    
    回答：
    """
)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm_deepseek,
    retriever=compression_retriever,
    memory=memory,
    # return_source_documents=True,
    output_key="answer",  # 明确指定存储到内存的键
    combine_docs_chain_kwargs={  # 替代chain_type_kwargs
        "prompt": QA_CHAIN_PROMPT
    },
    verbose=True, # 独立传递verbose参数
)


questions = [
    "什么是VMAX的安全加固？", 
    "安全加固的操作步骤？",  # 需记忆前一轮的"主要内容"
    "整理成中文表格"  # 需合并多轮信息
]

for question in questions:
    result = qa_chain({"question": question})
    print(f"问题：{question}")
    print(f"回答：{result['answer']}")
    # print(f"引用的来源：{result['source_documents'][0].metadata}")  # 显示来源文档
    print("对话历史：", memory.load_memory_variables({}))

## 5. 添加历史对话的记忆功能 - v2024

现在我们已经实现了通过上传本地知识文档，然后将他们保存到向量知识库，通过将查询问题与向量知识库的召回结果进行结合输入到 LLM 中，我们就得到了一个相比于直接让 LLM 回答要好得多的结果。在与语言模型交互时，你可能已经注意到一个关键问题 - **它们并不记得你之前的交流内容**。这在我们构建一些应用程序（如聊天机器人）的时候，带来了很大的挑战，使得对话似乎缺乏真正的连续性。这个问题该如何解决呢？


### 5.1 记忆（Memory）

在本节中我们将介绍 LangChain 中的储存模块，即如何将先前的对话嵌入到语言模型中的，使其具有连续对话的能力。我们将使用 `ConversationBufferMemory` ，它保存聊天消息历史记录的列表，这些历史记录将在回答问题时与问题一起传递给聊天机器人，从而将它们添加到上下文中。

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

关于更多的 Memory 的使用，包括保留指定对话轮数、保存指定 token 数量、保存历史对话的总结摘要等内容，请参考 langchain 的 Memory 部分的相关文档。

### 5.2 对话检索链（ConversationalRetrievalChain）

对话检索链（ConversationalRetrievalChain）在检索 QA 链的基础上，增加了处理对话历史的能力。

它的工作流程是:
1. 将之前的对话与新问题合并生成一个完整的查询语句。
2. 在向量数据库中搜索该查询的相关文档。
3. 获取结果后,存储所有答案到对话记忆区。
4. 用户可在 UI 中查看完整的对话流程。

这种链式方式将新问题放在之前对话的语境中进行检索，可以处理依赖历史信息的查询。并保留所有信
息在对话记忆中，方便追踪。

接下来让我们可以测试这个对话检索链的效果：

使用上一节中的向量数据库和 LLM ！首先提出一个无历史对话的问题“这门课会学习 Python 吗？”，并查看回答。

In [ ]:
from langchain.chains import ConversationalRetrievalChain

retriever=vectordb.as_retriever()

qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)
question = "什么是南瓜书？"
result = qa({"question": question})
print(result['answer'])

然后基于答案进行下一个问题“为什么这门课需要教这方面的知识？”：

In [ ]:
question = "南瓜书包含哪些内容？"
result = qa({"question": question})
print(result['answer'])

可以看到，LLM 它准确地判断了这方面的知识，指代内容是强化学习的知识，也就
是我们成功地传递给了它历史信息。这种持续学习和关联前后问题的能力，可大大增强问答系统的连续
性和智能水平。

# 脚本汇总

In [ ]:
def get_llm():
    return OllamaLLM(base_url='http://localhost:11434', model='deepseek-r1:1.5b', temperature=0.1)


def generate_response(input_text):
    llm = get_llm()
    output = llm.invoke(input_text)
    output_parser = StrOutputParser()
    return output_parser.invoke(output)

In [ ]:
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
import cohere
from langchain_community.vectorstores import Milvus


# 初始化函数 (保持你的原始代码不变)
# def get_vectordb():
#     emb_bgem3 = OllamaEmbeddings(base_url='http://localhost:11434', model="bge-m3:latest")
#     persist_directory = '../chroma-vmax'
#     vectordb = Chroma(
#         persist_directory=persist_directory,
#         collection_name="vmax-s",
#         embedding_function=emb_bgem3
#     )
#     return vectordb

# 初始化 Milvus 向量数据库
def get_vectordb():
    emb_bgem3 = OllamaEmbeddings(base_url='http://localhost:11434', model="bge-m3:latest")
    
    # Milvus 连接参数
    vectordb = Milvus(
        embedding_function=emb_bgem3,
        collection_name="Vmaxs",  # Milvus 集合名称
        connection_args={
            "host": "192.168.0.188",  # Milvus 服务器地址
            "port": "19530",      # Milvus 默认端口
        },
        # 可选：是否自动创建集合（如果不存在）
        # auto_create_collection=True,
    )
    return vectordb


def get_llm():
    return OllamaLLM(base_url='http://localhost:11434', model='deepseek-r1:1.5b', temperature=0.1, streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()])


def get_qa_chain_without_memory(question: str):
    vectordb = get_vectordb()
    myllm = get_llm()

    cohere_client = cohere.Client(api_key="Tahx1eySFbKvu9sTyTXrRLf59la3ZUG9vy02stRZ")
    compressor = CohereRerank(
        client=cohere_client,
        top_n=5,
        model="rerank-multilingual-v3.0"
    )

    base_retriever = vectordb.as_retriever(
        search_kwargs={"k": 15},
        search_type="mmr",
    )

    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=base_retriever
    )

    qa_chain = RetrievalQA.from_chain_type(
        llm=myllm,
        retriever=compression_retriever,
        return_source_documents=True,
        chain_type_kwargs={
            "prompt": PromptTemplate(
                input_variables=["context", "question"],
                template="""你是DeepSeek VMAX-S知识助手。使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
                案。总是在回答的最后说“谢谢你的提问！”。
                {context}
                问题: {question}
                """
            ),
        }
    )

    result = qa_chain({"query": question})
    return result["result"]


print(get_qa_chain_without_memory("介绍下VMAX的上网日志业务"))


In [ ]:
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
import cohere

# Initialize memory outside the function so it persists across questions
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# 初始化 Milvus 向量数据库
def get_vectordb():
    emb_bgem3 = OllamaEmbeddings(base_url='http://localhost:11434', model="bge-m3:latest")
    
    # Milvus 连接参数
    vectordb = Milvus(
        embedding_function=emb_bgem3,
        collection_name="Vmaxs",  # Milvus 集合名称
        connection_args={
            "host": "192.168.0.188",  # Milvus 服务器地址
            "port": "19530",      # Milvus 默认端口
        },
        # 可选：是否自动创建集合（如果不存在）
        # auto_create_collection=True,
    )
    return vectordb

def get_llm():
    return OllamaLLM(base_url='http://localhost:11434', model='deepseek-r1:1.5b', temperature=0.1)

def get_qa_chain_with_memory(question: str):
    vectordb = get_vectordb()
    myllm = get_llm()

    cohere_client = cohere.Client(api_key="Tahx1eySFbKvu9sTyTXrRLf59la3ZUG9vy02stRZ")
    compressor = CohereRerank(
        client=cohere_client,
        top_n=5,
        model="rerank-multilingual-v3.0"
    )

    base_retriever = vectordb.as_retriever(
        search_kwargs={"k": 15},
        search_type="mmr",
    )

    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=base_retriever
    )

    QA_CHAIN_PROMPT = PromptTemplate(
        input_variables=["chat_history", "question", "context"],
        template="""
        你是DeepSeek VMAX-S知识助手。使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。总是在回答的最后说“谢谢你的提问！
        
        历史对话：
        {chat_history}
        
        上下文：
        {context}
        
        问题：{question}
        
        回答：
        """
    )

    # QA_CHAIN_PROMPT = PromptTemplate(
    #     input_variables=["context","question"],
    #     template="""你是DeepSeek VMAX-S知识助手。使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。总是在回答的最后说“谢谢你的提问！”。
    #             {context}
    #             问题: {question}
    #     """
    # )

    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=get_llm(),
        retriever=compression_retriever,
        memory=memory,
        output_key="answer",
        combine_docs_chain_kwargs={
            "prompt": QA_CHAIN_PROMPT
        },
        verbose=True,
    )

    result = qa_chain({"question": question})  # Changed from "query" to "question"
    return result

questions = [
    "什么是VMAX的上网日志业务？",
    "上网日志业务包含哪些功能？",  # 需记忆前一轮的"主要内容"
    "整理成中文表格"  # 需合并多轮信息
]

for question in questions:
    result = get_qa_chain_with_memory(question)  # Pass string directly, not dict
    print(f"问题：{question}")
    print(f"回答：{result['answer']}")
    print("对话历史：", memory.load_memory_variables({}))
    print("\n" + "="*50 + "\n")

In [ ]:
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
import cohere
from langchain_community.vectorstores import Milvus
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Singleton pattern for vector database
_vectordb_instance = None

def get_vectordb():
    global _vectordb_instance
    if _vectordb_instance is None:
        try:
            emb_bgem3 = OllamaEmbeddings(base_url='http://localhost:11434', model="bge-m3:latest")
            
            _vectordb_instance = Milvus(
                embedding_function=emb_bgem3,
                collection_name="Vmaxs",
                connection_args={
                    "host": "192.168.0.188",
                    "port": "19530",
                },
                # Add consistency level if needed
                consistency_level="Strong",
            )
            logger.info("Successfully connected to Milvus vector database")
        except Exception as e:
            logger.error(f"Failed to connect to Milvus: {str(e)}")
            raise
    return _vectordb_instance

def get_llm():
    return OllamaLLM(
        base_url='http://localhost:11434', 
        model='deepseek-r1:1.5b', 
        temperature=0.1,
        # Add timeout parameters
        request_timeout=60
    )

def get_qa_chain_without_memory(question: str):
    try:
        vectordb = get_vectordb()
        myllm = get_llm()

        cohere_client = cohere.Client(api_key="Tahx1eySFbKvu9sTyTXrRLf59la3ZUG9vy02stRZ")
        compressor = CohereRerank(
            client=cohere_client,
            top_n=5,
            model="rerank-multilingual-v3.0"
        )

        # Milvus-specific search parameters
        base_retriever = vectordb.as_retriever(
            search_kwargs={
                "k": 15,
                # Add Milvus-specific search parameters if needed
                "params": {"nprobe": 10}
            },
            search_type="mmr",
        )

        compression_retriever = ContextualCompressionRetriever(
            base_compressor=compressor,
            base_retriever=base_retriever
        )

        qa_chain = RetrievalQA.from_chain_type(
            llm=myllm,
            retriever=compression_retriever,
            return_source_documents=True,
            chain_type_kwargs={
                "prompt": PromptTemplate(
                    input_variables=["context", "question"],
                    template="""你是DeepSeek VMAX-S知识助手。使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
                    案。总是在回答的最后说"谢谢你的提问！"。
                    {context}
                    问题: {question}
                    """
                ),
            }
        )

        result = qa_chain({"query": question})
        return result["result"]
    except Exception as e:
        logger.error(f"Error in QA chain: {str(e)}")
        return "抱歉，处理您的请求时出现错误。"

# Example usage
if __name__ == "__main__":
    print(get_qa_chain_without_memory("介绍下VMAX的上网日志业务"))

In [ ]:
import torch
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_community.document_compressors.rankllm_rerank import RankLLMRerank

torch.cuda.empty_cache()

compressor = RankLLMRerank(top_n=3, model="rank_zephyr")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

del compressor